# Java Threads and Concurrency Utilities

## Chapter - 8
## Additional Concurrency Utilities

Coverage of the concurrency utilities by introducing you to concurrent collections, atomic variables, the Fork/Join Framework, and completion services.


ArrayList, TreeSet, HashMap, and other classes that implement these interfaces
are not thread-safe. However, you can make them thread-safe by using the synchronized wrapper methods located in the java.util.Collections class. For example, you can pass an ArrayList instance to Collections.synchronizedList() to obtain a thread-safe variant of ArrayList.

The following list offers a short sample of concurrency-oriented collection types that you’ll find in the java.util.concurrent package:
* BlockingQueue is a subinterface of java.util.Queue that also supports blocking operations that wait for the queue to become nonempty before retrieving an element and wait for space to become available in the queue before storing an element. Each of the ArrayBlockingQueue, DelayQueue, LinkedBlockingQueue, PriorityBlockingQueue, and SynchronousQueue classes implements this interface directly. The LinkedBlockingDeque and LinkedTransferQueue classes implement this interface via BlockingQueue subinterfaces.
* ConcurrentMap is a subinterface of java.util.Map that declares additional indivisible putIfAbsent(), remove(), and replace() methods. The ConcurrentHashMap class (the concurrent equivalent of java.util.HashMap), the ConcurrentNavigableMap class, and the ConcurrentSkipListMap class implement this interface.

## Using BlockingQueue and ArrayBlockingQueue
BlockingQueue’s Javadoc reveals the heart of a producer-consumer application that’s vastly simpler than the equivalent application shown in Chapter 3

In [1]:
BlockingQueue<Character> bq;
bq = new ArrayBlockingQueue<Character>(1);
ExecutorService executor = Executors.newFixedThreadPool(2);

In [2]:
Runnable producer = () -> {
    for (char ch = 'A'; ch <= 'Z'; ch++) {
        try {
            bq.put(ch);
            System.out.printf("%c produced by " + "producer.%n", ch);
        } catch (InterruptedException ie) {
        }
    }
};

In [3]:
Runnable consumer = () -> {
    char ch = '\0';
    do {
        try {
            ch = bq.take();
            System.out.printf("%c consumed by " + "consumer.%n", ch);
        } catch (InterruptedException ie) {
        }
    } while (ch != 'Z');
};

In [4]:
executor.execute(producer);
executor.execute(consumer);
executor.shutdownNow();

A produced by producer.
A consumed by consumer.
B produced by producer.
B consumed by consumer.
C produced by producer.
C consumed by consumer.
D produced by producer.
D consumed by consumer.
E produced by producer.
E consumed by consumer.
F produced by producer.
F consumed by consumer.
G produced by producer.
G consumed by consumer.
H produced by producer.
H consumed by consumer.
I consumed by consumer.
I produced by producer.
J produced by producer.
J consumed by consumer.
K produced by producer.
L produced by producer.
K consumed by consumer.
L consumed by consumer.
M produced by producer.
M consumed by consumer.
N produced by producer.
N consumed by consumer.
O produced by producer.
O consumed by consumer.
P produced by producer.
P consumed by consumer.
Q consumed by consumer.
Q produced by producer.
R produced by producer.
S produced by producer.
R consumed by consumer.
S consumed by consumer.
T consumed by consumer.
T produced by producer.
U produced by producer.
V produced by pr

[]

In [5]:
public class BlockingQueueManMade<T> {
    int size;
    List<T> items;

    public BlockingQueueManMade(int size) {
        super();
        this.size = size;
        this.items = new ArrayList<>();
    }
    
    public synchronized void put(T item) throws InterruptedException {
        while(this.items.size() == this.size) {
            wait();
        }
        
        if(this.items.size() == 0) {
            notifyAll();
        }
        
        this.items.add(item);
    }
    
    public synchronized T take() throws InterruptedException{
        while(this.items.size() == 0) {
            wait();
        }
        
        if(this.items.size() == this.size) {
            notifyAll();
        }
        
        return this.items.remove(0);
    }
}

In [6]:
BlockingQueueManMade<Integer> blockingQueue = new BlockingQueueManMade<>(1);

In [11]:
Runnable producer = () -> {
    try {
        int iterator = 0;
        while (true) {
            if (iterator++ == 10)
                break;

            String name = Thread.currentThread().getName();
            Integer i = (int) (Math.random() * 10);
            blockingQueue.put(i);
            System.out.println(name + " Producing:-> " + i + " at " +System.nanoTime());
        }
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
};

Runnable consumer = () -> {
    try {
        int iterator = 0;
        while (true) {
            if (iterator++ == 10)
                break;
            String name = Thread.currentThread().getName();
            Integer take = blockingQueue.take();
            System.out.println(name + " Consuming:<- " + take + " at " +System.nanoTime());
        }
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
};

In [12]:
BlockingQueueManMade<Integer> blockingQueue = new BlockingQueueManMade<>(1);

Thread threadProducer = new Thread(producer);
Thread threadConsumer = new Thread(consumer);
final ExecutorService executor = Executors.newFixedThreadPool(10);

In [13]:
executor.execute(threadProducer);
executor.execute(threadConsumer);
executor.shutdown();

pool-3-thread-1 Producing:-> 9 at 28273718212769
pool-3-thread-1 Producing:-> 5 at 28273785024520
pool-3-thread-2 Consuming:<- 9 at 28273785020644
pool-3-thread-2 Consuming:<- 5 at 28273786240183
pool-3-thread-1 Producing:-> 8 at 28273786254857
pool-3-thread-2 Consuming:<- 8 at 28273786741479
pool-3-thread-1 Producing:-> 4 at 28273787178576
pool-3-thread-2 Consuming:<- 4 at 28273787498584
pool-3-thread-1 Producing:-> 3 at 28273787889204
pool-3-thread-2 Consuming:<- 3 at 28273788268090
pool-3-thread-1 Producing:-> 6 at 28273788593536
pool-3-thread-1 Producing:-> 1 at 28273789403084
pool-3-thread-2 Consuming:<- 6 at 28273789032306
pool-3-thread-2 Consuming:<- 1 at 28273790628345
pool-3-thread-2 Consuming:<- 6 at 28273790987216
pool-3-thread-1 Producing:-> 6 at 28273790837185
pool-3-thread-2 Consuming:<- 4 at 28273791943401
pool-3-thread-1 Producing:-> 4 at 28273791928104
pool-3-thread-2 Consuming:<- 4 at 28273792749777
pool-3-thread-1 Producing:-> 4 at 28273792736971


### Blocking queue using Lock and Condition

In [10]:
import java.util.LinkedList;
import java.util.Queue;
import java.util.concurrent.locks.Condition;
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

public class BlockingQueue<T> {
    private Queue<T> queue = new LinkedList<T>();
    private int capacity;
    private Lock lock = new ReentrantLock();
    private Condition notFull = lock.newCondition();
    private Condition notEmpty = lock.newCondition();

    public BlockingQueue(int capacity) {
        this.capacity = capacity;
    }

    public void put(T element) throws InterruptedException {
        lock.lock();
        try {
            while (queue.size() == capacity) {
                notFull.await();
            }
            queue.add(element);
            notEmpty.signal();
        } finally {
            lock.unlock();
        }
    }

    public T take() throws InterruptedException {
        lock.lock();
        try {
            while (queue.isEmpty()) {
                notEmpty.await();
            }
            T item = queue.remove();
            notFull.signal();
            return item;
        } finally {
            lock.unlock();
        }
    }
}